In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
   
    temperature=0,
    model_name="llama-3.3-70b-versatile",
    groq_api_key = "add your API key here"
  
)

response= llm.invoke("Tell me about India")
print(response.content)

India, officially known as the Republic of India, is a country located in South Asia. It is the seventh-largest country by land area, the second-most populous country, and the most populous democracy in the world. India is a diverse and vibrant nation with a rich history, culture, and natural beauty.

**Geography:**
India is bounded by the Himalayan Mountains to the north, the Arabian Sea to the west, the Bay of Bengal to the east, and the Indian Ocean to the south. The country has a diverse geography, with mountains, plateaus, plains, and coastal regions. The Ganges River, one of the most sacred rivers in Hinduism, flows through the northern part of the country.

**Climate:**
India has a varied climate, ranging from tropical in the south to temperate in the north. The country experiences four main seasons: winter (December to February), summer (March to May), monsoon (June to September), and post-monsoon (October to November).

**Culture:**
Indian culture is a rich and diverse tapestr

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/analyst-finance-emea/job/R-50715")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Analyst, Finance EMEA










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Languag

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [4]:
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Analyst, Finance EMEA',
 'experience': "Bachelor's degree or equivalent experience, preferably in a business-related field with emphasis in Finance, Business Administration and Accounting. Experience in Planning, Budgeting, and Financial modelling.",
 'skills': 'Excellent analytical and presentation skills, with the ability to translate sophisticated analyses into clear and compelling narratives. Ability to operate optimally in agile working environments communicating effectively with teammates while handling multiple priorities.',
 'description': "We're looking for a Financial Analyst, Finance Consumer & Product, for Nike EMEA (Europe, Middle East, and Africa) within the Consumer Finance team. This person will play a meaningful role in driving visibility in Nike EMEAs financials via the lens of product throughout product life cycles and financial forecast cycles."}

In [6]:
type(json_res)

dict

In [7]:
import pandas as pd

df= pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
links = collection.query(query_texts=['Experience in python','Experience in React'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [10]:
job= json_res
job['skills']

'Excellent analytical and presentation skills, with the ability to translate sophisticated analyses into clear and compelling narratives. Ability to operate optimally in agile working environments communicating effectively with teammates while handling multiple priorities.'

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Financial Analysis with AtliQ's Automated Solutions

Dear Hiring Manager,

I came across the job posting for an Analyst, Finance EMEA at Nike, and I was impressed by the role's focus on driving visibility in financials through product life cycles and financial forecast cycles. As a Business Development Executive at AtliQ, I believe our company can support Nike's financial analysis needs with our automated tools and tailored solutions.

At AtliQ, we have a proven track record of empowering enterprises with scalable, optimized, and efficient solutions. Our expertise in AI and software consulting can help streamline financial modeling, planning, and budgeting processes. We can develop customized tools to translate sophisticated analyses into clear and compelling narratives, aligning with your requirements for excellent analytical and presentation skills.

Our portfolio showcases our capabilities in machine learning and Python development, which can be leveraged to enhan